<a href="https://colab.research.google.com/github/lephuocdat2000/truyvandaphuongtien/blob/master/CapstoneProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
#đọc file data
import pandas as pd
import numpy as np
import math
import re
book_info=pd.read_json('/content/Book_info.json',encoding='utf-8')
book_info.head()

,NO,BIBID,AUTHOR,TITLE,PUBLISHER,PUBDATE,COUNTHOLDING,TOPIC,CONTENT
0,0,4386,Trần Hồng Nghi,Hướng dẫn thực hành phát triển ứng dụn...,Đại học Quốc gia Tp. Hồ Chí Min,2019,100,"[Mạng máy tính, Hệ điều hành Android]",Cung cấp những kiến thức cơ bản về công ...
1,1,4385,Lâm Đức Khải,Thiết kế luận lý số 2,Đại học Quốc gia Thành phố Hồ Chí Min,2019,100,[Kỹ thuật máy tính],None
2,2,4384,Cao Thị Nhạn,Phân tích thiết kế Hệ thống thông tin,Đại học Quốc gia Thành phố Hồ Chí Min,2019,100,[Hệ thống thông tin],None
3,3,4382,Đỗ Phúc,Phân tích Dữ liệu lớn,Đại học Quốc gia Thành phố Hồ Chí Min,2019,100,"[Hệ thống thông tin, Giáo trình]",None
4,4,4365,Nguyễn Anh Tuấn,Hệ điều hành Linux,Đại học Quốc gia TPHC,2019,100,[Hệ điều hành Linux],None


In [105]:
#Xây dựng tập từ điển 
def Dic_building(a):
  contents=book_info[a].to_numpy()
  lst_contents=[]
  dictionary=set()
  if (a!='TOPIC'):
    for content in contents:
       if content==None: continue
       words = content.replace('"', '').replace('.', '').replace("'","").split()
       lst_contents.append(words)
       dictionary.update(words)
  else:
    for content in contents:
       for i in content:
         words=i.replace('"', '').replace('.', '').replace("'","").split()
         lst_contents.append(words)
         dictionary.update(words)
  dictionary = list(dictionary)
  return contents,lst_contents,dictionary
#Xây dựng Inverted File
def build_inverted_files(dictionary,lst_contents):
   inv_files=dict()
   for k,word in enumerate(dictionary):
      inv_files[word] = set()
      for i,content in enumerate(lst_contents):
         if word in content:
            count=content.count(word)
            tf=count/len(content)
            tup_word_tf=(i,tf)
            inv_files[word].add(tup_word_tf)
   return inv_files

#Tính toán tf-idf
def calcu_word_tfidf(inv_files,lst_contents):
    tf_idf_arr=np.zeros(len(inv_files))
    total_content=len(lst_contents)
    for i,word in enumerate(inv_files):
        k=len(inv_files[word])
        idf=math.log(total_content/k)
        for tup in inv_files[word]:
            temp=list(tup)
            temp[1]*=idf
            tup=tuple(temp)
    return inv_files
#Xây dựng inverted file theo topic
def Build_invfile(a):
   contents,lst_contents,dictionary = Dic_building(a)
   inv_files = build_inverted_files(dictionary,lst_contents)
   inv_files = calcu_word_tfidf(inv_files,lst_contents)
   return contents,lst_contents,dictionary,inv_files

In [95]:
contents_TOPIC,lst_contents_TOPIC,dic_TOPIC,inv_files_TOPIC, = Build_invfile('TOPIC')
contents_TITLE,lst_contents_TITLE,dic_TITLE,inv_files_TITLE = Build_invfile('TITLE')
contents_AUTHOR,lst_contents_AUTHOR,dic_AUTHOR,inv_files_AUTHOR = Build_invfile('AUTHOR')
contents_CONTENT,lst_contents_CONTENT,dic_CONTENT,inv_files_CONTENT = Build_invfile('CONTENT')


{'máy': {(59, 0.25), (0, 0.3333333333333333), (39, 0.25), (62, 0.25), (64, 0.25), (72, 0.25), (2, 0.25), (69, 0.25), (60, 0.25), (74, 0.5), (24, 0.2), (77, 0.25)}, 'đại': {(88, 0.25)}, 'nghiệp': {(43, 0.5)}, 'language': {(41, 0.5)}, 'trẻ': {(23, 0.16666666666666666)}, 'viên': {(18, 0.058823529411764705)}, 'di': {(116, 0.14285714285714285)}, 'Kỹ': {(59, 0.25), (2, 0.25), (72, 0.25), (60, 0.25)}, 'diễn': {(65, 0.25), (68, 0.25)}, 'tri': {(67, 0.25), (66, 0.25), (65, 0.25), (68, 0.25)}, 'thực': {(96, 0.16666666666666666), (103, 0.16666666666666666), (94, 0.16666666666666666), (140, 0.16666666666666666), (102, 0.16666666666666666), (120, 0.16666666666666666), (86, 0.16666666666666666), (109, 0.16666666666666666), (132, 0.16666666666666666)}, 'kê': {(35, 0.5), (129, 0.25)}, 'Tri': {(113, 0.5)}, 'cho': {(18, 0.058823529411764705)}, 'nhiên': {(40, 0.25)}, 'CSDL': {(7, 1.0)}, 'Biểu': {(65, 0.25), (68, 0.25)}, 'điện': {(114, 0.25), (73, 0.25), (80, 0.5)}, 'Điện': {(81, 0.5), (159, 0.5

In [112]:
#Tính toán tfidf cho query 
def Create_tfidf_query(query,lst_contents,inv_files):
   query_words=re.findall("(\w+)", query)
   dict_query=set()
   dict_query.update(query_words)
   dict_query=list(dict_query)
   tfidf_query=np.zeros(len(dict_query))
   total_content=len(lst_contents)
   for i,word in enumerate(dict_query):
     try:
        idf=1+np.log(total_content/len(inv_files[word]))
     except: 
        idf=1+np.log(total_content)
     tfidf_query[i]=query_words.count(word)*idf/len(query_words)
   return tfidf_query,dict_query

#Thêm phần tử và kiểm tra
def Add_arr(arrs,arr):
   appear=False
   for i in arrs:
      if i[0]==arr[0]: 
        appear=True
        break
   if appear==False: arrs.append(arr)
   else: 
      for i in range(len(arrs)):
           if arrs[i][0]==arr[0]: arrs[i][1]+=[1]
   return arrs
def swap(a,b):
    temp=a
    a=b
    b=temp
    return a,b
def Sort_arr(arr):
   for i in range(0,len(arr)-1):
        for j in range(i+1,len(arr)):
           if (arr[j][1]>arr[i][1]): 
               arr[i][0],arr[j][0]=swap(arr[i][0],arr[j][0])
               arr[i][1],arr[j][1]=swap(arr[i][1],arr[j][1])
   return arr
#Mô hình tích vô hướng
def ScalarModel(tfidf_query,dict_query,inv_files,dic):
    #Tạo mảng chứa kết quả của tính vô hướng và sắp xếp
    kq_scalar=[]
    print(inv_files)
    for i,word in  enumerate(dict_query):
        if word in dic:
           print(word,inv_files[word])
           for tup in inv_files[word]:
              tempp=list(tup)
              tempp[1]=tempp[1]*tfidf_query[i]
              kq_scalar=Add_arr(kq_scalar,tempp)

    kq_scalar=Sort_arr(kq_scalar)
    arr_scalar=[]
    for i in range(len(kq_scalar)):
        arr_scalar.append(kq_scalar[i][0])
    return arr_scalar

#Search
def Search(query,topic):
   contents,lst_contents,dic,inv_files=Build_invfile(topic)
   tfidf_query,dict_query=Create_tfidf_query(query,lst_contents,inv_files)
   scalar=ScalarModel(tfidf_query,dict_query,inv_files,dic)
   return scalar
query=str(input())
scalar=Search(query,'TITLE')
print(scalar)


Phân tích và thiết
{'ngôn': {(19, 0.07692307692307693), (26, 0.1111111111111111), (41, 0.14285714285714285)}, 'máy': {(41, 0.14285714285714285), (39, 0.5), (35, 0.07692307692307693), (81, 0.25), (72, 0.3333333333333333), (47, 0.25)}, 'rạc': {(79, 0.3333333333333333)}, '(Nosol': {(15, 0.0625)}, 'đại': {(51, 0.25), (45, 0.14285714285714285)}, 'nghiệp': {(27, 0.125)}, 'khảo': {(15, 0.0625)}, 'căn': {(48, 0.09090909090909091)}, 'trẻ': {(10, 0.07142857142857142), (18, 0.16666666666666666)}, 'di': {(8, 0.125), (58, 0.07692307692307693), (0, 0.07692307692307693)}, 'thương': {(57, 0.09090909090909091)}, 'về': {(11, 0.2), (32, 0.07692307692307693)}, 'diễn': {(21, 0.125), (37, 0.1)}, 'tri': {(56, 0.16666666666666666), (37, 0.1), (20, 0.1111111111111111)}, 'thực': {(59, 0.06666666666666667), (73, 0.07142857142857142), (0, 0.07692307692307693), (30, 0.08333333333333333), (53, 0.09090909090909091), (44, 0.1111111111111111), (35, 0.07692307692307693), (58, 0.07692307692307693), (49, 0.1), (